# importing required modules

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score,KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.naive_bayes import MultinomialNB,BernoulliNB,GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC,SVR
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
# importing ensembling algorithms
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier,BaggingClassifier
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor,RadiusNeighborsClassifier
from sklearn.decomposition import PCA,TruncatedSVD

# Reading the data

In [ ]:
data=pd.read_csv('../input/top50spotify2019/top50.csv',encoding='ISO-8859-1')
data.head()

In [ ]:
data.reset_index(drop=True)

In [ ]:
print(data.info())

# Processing of the data given

In [ ]:
sns.heatmap(data.isnull())

In [ ]:
data.isnull().sum()

#### there is nothing to process as the data doesn't have any null values

In [ ]:
# finding the number of songs of each genre
data.groupby(data['Genre']).size()

In [ ]:
# the columns names are not well represented so we'll rename our columns
data.rename(columns={'Unnamed: 0':'sl.No','Track.Name':'Track_name','Artist.Name':'Artist_name','Beats.Per.Minute':'Beats_per_minute','Loudness..dB..':'Loudness(db)','Valence.':'Valence','Length.':'Length','Acousticness..':'Acousticness','Speechiness.':'Speechiness'},inplace=True)
data.head()

In [ ]:
# finding the number of songs sung by each Artist
data.groupby(data['Artist_name']).size()

In [ ]:
# Artist with most popular track
data['Artist_name'][data['Popularity']==data['Popularity'].max()]

In [ ]:
# plotting between Genre and count
sns.countplot(data=data,y='Genre')

In [ ]:
# popularity of each and every track_name
plt.pie(np.array(data['Popularity']),labels=np.array(data.Track_name),radius=10,autopct='%0.1f%%',shadow=True,startangle=90)
plt.show()

In [ ]:
# Danceability of each and every track_name
plt.pie(np.array(data['Danceability']),labels=np.array(data.Track_name),radius=10,autopct='%0.1f%%',shadow=True,startangle=90,explode=[0,0.5,0,0,1.0,0,0.5,0,0.5,0,0,0,0.5,0.5,0,0,1.0,0.7,0,0,0.8,0,0,1.5,0.5,0,1.0,0,0.5,0,0,1.0,0,0,0,1.0,1.0,1.0,2.0,0,0,0,0,0,0,1.0,0.5,0,0,0.5])
plt.show()

In [ ]:
# representing the data in a pairplot()
sns.pairplot(data)

In [ ]:
# countplot representing total no. of songs by a group of artists
sns.countplot(data=data,y=data['Artist_name'].value_counts())

In [ ]:
fig=plt.figure(figsize=(10,30))
sns.barplot(data=data,y='Artist_name',x=data['Artist_name'].nunique())

In [ ]:
#barplot of each numerical column in the data
fig=plt.figure(figsize=(20,20))
ax=plt.gca()
data.plot(kind='hist',subplots=True,ax=ax)
plt.show()

In [ ]:
fig=plt.figure(figsize=(20,20))
ax=plt.gca()
data.plot(kind='kde',subplots=True,ax=ax)
plt.show()

In [ ]:
# finding correlation of the data
fig=plt.figure(figsize=(10,10))
sns.heatmap(data.corr())

In [ ]:
fig=plt.figure(figsize=(20,20))
ax=plt.gca()
data.plot(kind='box',subplots=True,ax=ax)
plt.show()

In [ ]:
# we will check the popularity based on danceability,acousticness and energy 
fig=plt.figure(figsize=(10,10))
plt.subplot(3,1,1)
plt.xlabel('popularity')
plt.ylabel('energy')
plt.title('popularity vs energy')
sns.barplot(data=data,y='Energy',x='Popularity')
plt.subplot(3,1,2)
plt.xlabel('popularity')
plt.ylabel('danceability')
plt.title('popularity vs danceability')
sns.barplot(data=data,y='Danceability',x='Popularity')
plt.subplot(3,1,3)
plt.xlabel('popularity')
plt.ylabel('acoustics')
plt.title('popularity vs acousticness')
sns.barplot(data=data,y='Acousticness',x='Popularity')



In [ ]:
# relation between energy and loudness(db)
fig=plt.figure(figsize=(10,10))
sns.regplot(data=data,x='Energy',y='Loudness(db)')

In [ ]:
# relation between acousticness and loudness
fig=plt.figure(figsize=(10,10))
sns.jointplot(data=data,x='Acousticness',y='Loudness(db)',kind='kde',color='black')

In [ ]:
fig=plt.figure(figsize=(15,15))
sns.countplot(data=data,y='Popularity',hue='Genre')
plt.legend(loc="upper right")

In [ ]:
data.columns

In [ ]:
# applying some machine learning modelling algorithms
lin_reg=LinearRegression()
X=data.loc[:,['Beats_per_minute',
       'Energy', 'Danceability', 'Loudness(db)', 'Liveness', 'Valence',
       'Length', 'Acousticness', 'Speechiness']]
y=data.loc[:,['Popularity']]
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)
lin_reg.fit(X_train,y_train)

In [ ]:
y_pred=lin_reg.predict(X_test)
y_pred,y_test

In [ ]:
print(lin_reg.score(X_train,y_train))
lin_reg.score(X_test,y_pred)

In [ ]:
plt.subplot(2,2,1)
sns.kdeplot(X_train,y_train)
plt.subplot(2,2,2)
sns.kdeplot(X_test,y_pred)
plt.subplot(2,2,3)
plt.scatter(X_train.iloc[:,0:1],y_train)
plt.plot(X_test.iloc[:,0:1],y_pred)

In [ ]:
# comparing accuracy scores for LinearRegression and Lasso and Ridge
lin_accuracy=lin_reg.score(X_test,y_pred)
ridge=Ridge().fit(X_train,y_train)
ridge_accuracy=ridge.score(X_test,ridge.predict(X_test))
lasso=Lasso().fit(X_train,y_train)
lasso_accuracy=lasso.score(X_test,lasso.predict(X_test))
print(lin_accuracy)
print(ridge_accuracy)
lasso_accuracy

#### all the three linear regression shows similar accuracy scores

In [ ]:
# predicting the genre by taking the numerical values present in the dataset
# - we will use KNeighborsClassifier Algorithm
X=data.loc[:,['Beats_per_minute',
       'Energy', 'Danceability', 'Loudness(db)', 'Liveness', 'Valence',
       'Length', 'Acousticness', 'Speechiness','Popularity']]
y=data.loc[:,['Genre']]
knn_clf=KNeighborsClassifier(n_neighbors=5)
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)
knn_clf.fit(X_train,y_train)
print(knn_clf.score(X_train,y_train))
knn_clf.score(X_test,knn_clf.predict(X_test))

In [ ]:
n_range=range(1,20)
accuracy=[]
for n in n_range:
    knn_clf=KNeighborsClassifier(n_neighbors=n)
    knn_clf.fit(X_train,y_train)
    accuracy.append(knn_clf.score(X_test,knn_clf.predict(X_test)))
plt.scatter(n_range,accuracy)

In [ ]:
print(knn_clf.predict(X_test))

In [ ]:
gnb=GaussianNB()
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)
print(gnb.fit(X_train,y_train))
print(gnb.score(X_train,y_train))
gnb.score(X_test,gnb.predict(X_test))

In [ ]:
gnb.predict(X_test)

### if we see the above observations GaussianNB algorithm shows some good results compared to KNeighborsClassifier algorithm


# we will use PCA technique to reduce the dimensionality of the data given

In [ ]:
# we will have to first import PCA from decomposition package
from sklearn.decomposition import PCA
pca=PCA(n_components=5)
pca

In [ ]:
# also we will have to scale the data into standard scaler
scaler=StandardScaler()
scaled_X=scaler.fit_transform(X)
pca_scaled=pca.fit_transform(scaled_X)
pca_scaled

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(pca_scaled,y,random_state=0,test_size=0.2)
print(gnb.fit(X_train,y_train))
print(gnb.score(X_train,y_train))
gnb.score(X_test,gnb.predict(X_test))

In [ ]:
# we will find the correlation of the PCA scaled values
print(pd.DataFrame(pca_scaled).corr())
sns.heatmap(pd.DataFrame(pca_scaled).corr())

In [ ]:
sns.pairplot(pd.DataFrame(pca_scaled))

In [ ]:
sns.pairplot(pd.DataFrame(pca_scaled),corner=True)

In [ ]:
sns.pairplot(pd.DataFrame(pca_scaled),kind='kde')

In [ ]:
dec_clf=DecisionTreeClassifier(criterion='entropy')
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)
print(dec_clf.fit(X_train,y_train))
print(dec_clf.score(X_train,y_train))
dec_clf.score(X_test,dec_clf.predict(X_test))

In [ ]:
from sklearn import tree
fig=plt.figure(figsize=(20,20))
_=tree.plot_tree(dec_clf,feature_names=data.Popularity,class_names=data.Genre,filled=True,max_depth=3)